In [ ]:
# Define the forward-propagation algorithm: mnist_inference.py

# -*- coding: utf-8 -*-
import tensorflow as tf

INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

# Get variables by tf.get_variable. Creating variables during 
# training periods; loading those variables values by saved model when testing.
# Because we can rename the moving average variables when 
#loading variables, using variables when training by same name, 
#and using moving average values of variables when testing.
def get_weight_variable(shape,regularizer):
    weights = tf.get_variable("weights",shape,initializer=tf.truncated_normal_initializer(stddev=0.1))
    # When we have regularizer, adding the loss with regularization 
    # of current variables into the collection-losses.
    if regularizer != None:
        tf.add_to_collection('losses',regularizer(weights))
    return weights

def inference(input_tensor,regularizer):
    with tf.variable_scope('layer1'):
        weights = get_weight_variable([INPUT_NODE,LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases",[LAYER1_NODE],initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weights)+biases)
        
    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER1_NODE,OUTPUT_NODE],regularizer)
        biases = tf.get_variable("biases",[OUTPUT_NODE],initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1,weights) + biases
        
    return layer2

In [ ]:
# mnist_train.py

# -*- coding:utf-8 -*-
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import mnist_inference

# Parameters
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99

# Model saving path and name
MODEL_SAVE_PATH = "/PATH/TO/MODEL/"
MODEL_NAME = "MODEL.ckpt"

def train(mnist):
    x = tf.placeholder(tf.float32,[None,mnsit_inference.INPUT_NODE],name="x-input")
    y_ = tf.placeholder(tf.float32,[None,mnist_inference.OUTPUT_NODE],name="y-input")
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    
    y = mnist_inference.inference(x,regularizer)
    global_step = tf.Variable(0,trainable=False)
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y,tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,
                                               global_step,
                                              mnist.train.num_examples/BATCH_SIZE,
                                              LEARNING_RATE_DECAY)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate)\
                 .minimize(loss,global_step=global_step)
    
    with tf.control_dependencies([train_step,variables_averages_op]):
        train_op = tf.no_op(name='train')
        
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        for i in range(TRAINING_STEPS):
            batch_x,batch_y = mnist.train.next_batch(BATCH_SIZE)
            _,loss_values,step = sess.run([train_op,loss,global_step],
                                         feed_dict={x:batch_x,y_:batch_y})
            
            if i%1000 == 0:
                print("After %d training steps, loss on training batch is %f." % (step, loss_value))
                
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=global_step)
                
def main(argv=None):
    mnist = input_data.read_data_sets("./mnist",one_hot=True)
    train(mnist)
    
if __name__=='__main__':
    tf.app.run()

In [ ]:
# mnist_eval.py

# -*- coding: utf-8 -*-
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import mnist_inference
import mnist_train

# Loading a newest model per 10 secs, and testing accuracy of test data on it
EVAL_INTERVAL_SECS = 10

def evaluate(mnist):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32,[None,mnist_inference.INPUT_NODE],name='x-input')
        y_ = tf.placeholder(tf.float32,[None,mnist_inference.OUTPUT_NODE],name='y-input')
        
        validation_feed = {x:mnist.validation.images,
                          y_:mnist.validation.labels}
        # Set regularizer to None, cuz we don't care the loss with regularization when testing
        y = mnist_inference.inference(x,None)
        
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        
        # Loading model with changing the name of variables
        variable_averages = tf.train.ExponentialMovingAverage(mnist_train.MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)
        
        while True:
            with tf.Session() as sess:
                # tf.train.get_checkpoint_state can automatically find file name of the newest model in the file of checkpoint
                ckpt = tf.train.get_checkpoint_state(mnist_train.MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess,ckpt.model_checkpoint_path)
                    
                    # Retrive the iteration of saving model by file name
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    
                    accuracy_score = sess.run(accuracy,feed_dict=validation_feed)
                    
                    print("After %d training steps, validation accuracy is %f." % (global_step,accuracy_score))
                else:
                    print('No checkpoint file found')
                    return
                time.sleep(EVAL_INTERVAL_SECS)
                
def main(argv=None):
    mnist = input_data.read_data_sets("./mnist", one_hot=True)
    
if __name__=='__main__':
    tf.app.run()